### **In this example, we'll create a vector db based on a users/schemas views/tables of oracle database.**
 - **Retrieve all the views/tables from a oracle db user.**
 - **Create a documents list by reading those views and associated columns.**
 - **Create a vector DB over the document list using an embedding model.**
 - **Check the results by vector similarity search.**

In [111]:
import oracledb
from typing import List
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
import os
from dotenv import load_dotenv

In [112]:
load_dotenv()

True

In [113]:
# Best practice: create a secrete file and keep all the credentials there.
DB_CONFIG = {
    "user": 'aitest',
    "password": "aitest",
    "dsn": "localhost:1521/orcl.amernitech.com",
    "min": 2,
    "max": 5,
    "increment": 1    
}

In [114]:
# Define the connection pool for oracle db connection.
_pool = None

def init_pool():
    global _pool
    if not _pool:
        _pool = oracledb.create_pool(
            user=DB_CONFIG["user"],
            password=DB_CONFIG["password"],
            dsn=DB_CONFIG["dsn"],
            min=DB_CONFIG["min"],
            max=DB_CONFIG["max"],
            increment=DB_CONFIG["increment"],
            getmode=oracledb.SPOOL_ATTRVAL_WAIT,
            timeout=10
        )
    return _pool

In [115]:
# Write the query to retrieve the schemas details
schemas_query = "SELECT table_name, column_name FROM user_tab_columns where table_name not in ('VW_USERINFO','VW_USER_MENU','VW_USER_PROGRAM','USERS','ADD_EXAMPLES','NEW_VIEW_ADD') ORDER BY table_name, column_id"

In [116]:
def fetch_schema_docs() -> List[Document]:
    pool = init_pool()
    with pool.acquire() as conn:
        with conn.cursor() as cursor:
            cursor.execute(schemas_query)
            rows = cursor.fetchall()
            grouped = {}
            for table, column in rows:
                grouped.setdefault(table, []).append(column)
            return [Document(page_content=f"Table {t}: {', '.join(cols)}") for t, cols in grouped.items()]

In [117]:
fetch_schema_docs()

[Document(metadata={}, page_content='Table VW_EMPLOYEE_OUTSTANDING: COMPCODE, EMPLOYEE_CODE, EMPLOYEE_NAME, DLC_OUTSTANDING, GEN_OUTSTANDING, TOTAL_OUTSTANDING'),
 Document(metadata={}, page_content='Table VW_FM_NRTDETAILS: COMPCODE, NRTNUMBR, TAXICODE, VEHNUMBR, REGNNUMB, REASONCD, NRSTDATE, NRENDATE, DRIVERID, CLOSE_STATUS, OPEN_STATUS'),
 Document(metadata={}, page_content='Table VW_OUTSOURCE_WORKORDER: COMPCODE, DOCNUMBR, DOCTDATE, STATUSCD, Status_Desc, OPEN_STATUS, CLOSE_STATUS, TRANSFER_STATUS'),
 Document(metadata={}, page_content='Table VW_TAXI_COLLECTION: COMPCODE, CASHIER_NAME, PAID_DATE, DRIVER_ID, LOCATION_NAME, AMOUNT, TAXI_REVENUE, CALLTRIP, SURCHARGE_ITC, SURCHARGE_DRIVER, CREDIT_CARD_AMOUNT, ACTUAL_REVENUE, DRIVER_INCOME, SHFTCODE')]

In [118]:
# Define the embedding model
def get_embedding_model():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={'device': 'cpu'})

In [119]:
# Set the vector db path
VECTOR_DB_PATH = "data/schema_store.faiss"

In [120]:
# Define the function to build or load the vector db
def build_or_load_schema_vectordb():
    if not os.path.exists(VECTOR_DB_PATH):
        schema_docs = fetch_schema_docs()
        db = FAISS.from_documents(schema_docs, get_embedding_model())
        db.save_local(VECTOR_DB_PATH)
        return db
    else:
        return FAISS.load_local(VECTOR_DB_PATH, get_embedding_model(), allow_dangerous_deserialization=True)

In [121]:
# Load vector db
schema_vectorstore = build_or_load_schema_vectordb()

In [122]:
#user_input = "show the total open NRT"
user_input = "show the total taxi revenue"
#user_input = "show the total open outsource work orders"

In [123]:
schema_docs = schema_vectorstore.similarity_search(user_input, k=3)
schema_hint = "\n".join([doc.page_content for doc in schema_docs])

In [124]:
schema_docs

[Document(id='9b0c378d-f49e-46e1-8a96-8dfaef2f88e5', metadata={}, page_content='Table VW_TAXI_COLLECTION (It contains the taxi or driver collecton and driver revenue information as date wise.\nSo, for a specific range we can get the revenue.): COMPCODE, CASHIER_NAME, PAID_DATE, DRIVER_ID, LOCATION_NAME, AMOUNT, TAXI_REVENUE, CALLTRIP, SURCHARGE_ITC, SURCHARGE_DRIVER, CREDIT_CARD_AMOUNT, ACTUAL_REVENUE, DRIVER_INCOME, SHFTCODE'),
 Document(id='c5a84ebf-6e88-463d-b6af-6550ebdb137f', metadata={}, page_content="Table VW_FM_NRTDETAILS (It contains all the vehicle/taxi nrt details. \nSome of columns are flag based, like below.\nopen_status = 'Y': gives only opened nrt list.\nclose_status = 'Y': gives only closed nrt list.  \n): COMPCODE, NRTNUMBR, TAXICODE, VEHNUMBR, REGNNUMB, REASONCD, NRSTDATE, NRENDATE, DRIVERID, CLOSE_STATUS (close_status = 'Y': gives the close nrt list.), OPEN_STATUS (open_status = 'Y': gives the active/opened nrt list.)"),
 Document(id='28538f99-2d1e-4005-b7b3-84163b8b

In [125]:
schema_hint

"Table VW_TAXI_COLLECTION (It contains the taxi or driver collecton and driver revenue information as date wise.\nSo, for a specific range we can get the revenue.): COMPCODE, CASHIER_NAME, PAID_DATE, DRIVER_ID, LOCATION_NAME, AMOUNT, TAXI_REVENUE, CALLTRIP, SURCHARGE_ITC, SURCHARGE_DRIVER, CREDIT_CARD_AMOUNT, ACTUAL_REVENUE, DRIVER_INCOME, SHFTCODE\nTable VW_FM_NRTDETAILS (It contains all the vehicle/taxi nrt details. \nSome of columns are flag based, like below.\nopen_status = 'Y': gives only opened nrt list.\nclose_status = 'Y': gives only closed nrt list.  \n): COMPCODE, NRTNUMBR, TAXICODE, VEHNUMBR, REGNNUMB, REASONCD, NRSTDATE, NRENDATE, DRIVERID, CLOSE_STATUS (close_status = 'Y': gives the close nrt list.), OPEN_STATUS (open_status = 'Y': gives the active/opened nrt list.)\nTable VW_EMPLOYEE_OUTSTANDING (It contains employees or drivers payment outstanding or pending information.\nSome columns definition as mentioned below.\nDLC_Outstanding = Driving Licence Outstanding \nGEN_Out

## Example-2: Adding metadata(table comments, columns comments)

In [127]:
# Write the query to retrieve the schemas details along with table and columns comments
schemas_query = "SELECT nvl2(vcoment.comments,vcoment.table_name||' ('||vcoment.comments||')',vcoment.table_name) table_name,nvl2(vcc.comments,vcc.column_name||' ('||vcc.comments||')',vcc.column_name) column_name FROM user_tab_comments vcoment, user_tab_columns vcol, user_col_comments vcc Where vcoment.table_type = 'VIEW' And vcoment.table_name not in ('ADD_EXAMPLES','NEW_VIEW_ADD','USERS','VW_USERINFO','VW_USER_MENU','VW_USER_PROGRAM') And vcoment.table_name = vcol.table_name And vcol.table_name = vcc.table_name And vcol.column_name = vcc.column_name Order by vcoment.table_name, vcol.column_id"

In [128]:
def fetch_schema_docs_metadata() -> List[Document]:
    pool = init_pool()
    with pool.acquire() as conn:
        with conn.cursor() as cursor:
            cursor.execute(schemas_query)
            rows = cursor.fetchall()
            grouped = {}
            for table, column in rows:
                grouped.setdefault(table, []).append(column)
            return [Document(page_content=f"Table {t}: {', '.join(cols)}") for t, cols in grouped.items()]

In [129]:
fetch_schema_docs_metadata()

[Document(metadata={}, page_content='Table VW_EMPLOYEE_OUTSTANDING (It contains employees or drivers payment outstanding or pending information.\nSome columns definition as mentioned below.\nDLC_Outstanding = Driving Licence Outstanding \nGEN_Outstanding = General Outstanding\nTotal_Outstanding = Total Outstanding\n): COMPCODE, EMPLOYEE_CODE, EMPLOYEE_NAME, DLC_OUTSTANDING, GEN_OUTSTANDING, TOTAL_OUTSTANDING'),
 Document(metadata={}, page_content="Table VW_FM_NRTDETAILS (It contains all the vehicle/taxi nrt details. \nSome of columns are flag based, like below.\nopen_status = 'Y': gives only opened nrt list.\nclose_status = 'Y': gives only closed nrt list.  \n): COMPCODE, NRTNUMBR, TAXICODE, VEHNUMBR, REGNNUMB, REASONCD, NRSTDATE, NRENDATE, DRIVERID, CLOSE_STATUS (close_status = 'Y': gives the close nrt list.), OPEN_STATUS (open_status = 'Y': gives the active/opened nrt list.)"),
 Document(metadata={}, page_content="Table VW_OUTSOURCE_WORKORDER (It contains all the outsource workorders

In [130]:
# Define the function to build or load the vector db
def build_or_load_schema_vectordb():
    if not os.path.exists(VECTOR_DB_PATH):
        schema_docs = fetch_schema_docs_metadata()
        db = FAISS.from_documents(schema_docs, get_embedding_model())
        db.save_local(VECTOR_DB_PATH)
        return db
    else:
        return FAISS.load_local(VECTOR_DB_PATH, get_embedding_model(), allow_dangerous_deserialization=True)

In [131]:
# Load Vector DB
schema_vectorstore = build_or_load_schema_vectordb()

In [132]:
#user_input = "show the total open NRT"
user_input = "show the total taxi revenue"
#user_input = "show the total open outsource work orders"

In [133]:
schema_docs = schema_vectorstore.similarity_search(user_input, k=3)
schema_hint = "\n".join([doc.page_content for doc in schema_docs])

In [134]:
schema_docs

[Document(id='9b0c378d-f49e-46e1-8a96-8dfaef2f88e5', metadata={}, page_content='Table VW_TAXI_COLLECTION (It contains the taxi or driver collecton and driver revenue information as date wise.\nSo, for a specific range we can get the revenue.): COMPCODE, CASHIER_NAME, PAID_DATE, DRIVER_ID, LOCATION_NAME, AMOUNT, TAXI_REVENUE, CALLTRIP, SURCHARGE_ITC, SURCHARGE_DRIVER, CREDIT_CARD_AMOUNT, ACTUAL_REVENUE, DRIVER_INCOME, SHFTCODE'),
 Document(id='c5a84ebf-6e88-463d-b6af-6550ebdb137f', metadata={}, page_content="Table VW_FM_NRTDETAILS (It contains all the vehicle/taxi nrt details. \nSome of columns are flag based, like below.\nopen_status = 'Y': gives only opened nrt list.\nclose_status = 'Y': gives only closed nrt list.  \n): COMPCODE, NRTNUMBR, TAXICODE, VEHNUMBR, REGNNUMB, REASONCD, NRSTDATE, NRENDATE, DRIVERID, CLOSE_STATUS (close_status = 'Y': gives the close nrt list.), OPEN_STATUS (open_status = 'Y': gives the active/opened nrt list.)"),
 Document(id='28538f99-2d1e-4005-b7b3-84163b8b

In [135]:
for doc in schema_docs:
    print(doc.page_content, sep="\n")

Table VW_TAXI_COLLECTION (It contains the taxi or driver collecton and driver revenue information as date wise.
So, for a specific range we can get the revenue.): COMPCODE, CASHIER_NAME, PAID_DATE, DRIVER_ID, LOCATION_NAME, AMOUNT, TAXI_REVENUE, CALLTRIP, SURCHARGE_ITC, SURCHARGE_DRIVER, CREDIT_CARD_AMOUNT, ACTUAL_REVENUE, DRIVER_INCOME, SHFTCODE
Table VW_FM_NRTDETAILS (It contains all the vehicle/taxi nrt details. 
Some of columns are flag based, like below.
open_status = 'Y': gives only opened nrt list.
close_status = 'Y': gives only closed nrt list.  
): COMPCODE, NRTNUMBR, TAXICODE, VEHNUMBR, REGNNUMB, REASONCD, NRSTDATE, NRENDATE, DRIVERID, CLOSE_STATUS (close_status = 'Y': gives the close nrt list.), OPEN_STATUS (open_status = 'Y': gives the active/opened nrt list.)
Table VW_EMPLOYEE_OUTSTANDING (It contains employees or drivers payment outstanding or pending information.
Some columns definition as mentioned below.
DLC_Outstanding = Driving Licence Outstanding 
GEN_Outstanding = 

In [136]:
schema_hint

"Table VW_TAXI_COLLECTION (It contains the taxi or driver collecton and driver revenue information as date wise.\nSo, for a specific range we can get the revenue.): COMPCODE, CASHIER_NAME, PAID_DATE, DRIVER_ID, LOCATION_NAME, AMOUNT, TAXI_REVENUE, CALLTRIP, SURCHARGE_ITC, SURCHARGE_DRIVER, CREDIT_CARD_AMOUNT, ACTUAL_REVENUE, DRIVER_INCOME, SHFTCODE\nTable VW_FM_NRTDETAILS (It contains all the vehicle/taxi nrt details. \nSome of columns are flag based, like below.\nopen_status = 'Y': gives only opened nrt list.\nclose_status = 'Y': gives only closed nrt list.  \n): COMPCODE, NRTNUMBR, TAXICODE, VEHNUMBR, REGNNUMB, REASONCD, NRSTDATE, NRENDATE, DRIVERID, CLOSE_STATUS (close_status = 'Y': gives the close nrt list.), OPEN_STATUS (open_status = 'Y': gives the active/opened nrt list.)\nTable VW_EMPLOYEE_OUTSTANDING (It contains employees or drivers payment outstanding or pending information.\nSome columns definition as mentioned below.\nDLC_Outstanding = Driving Licence Outstanding \nGEN_Out

#### **Addition filter to find out only the relevant schemas**
 - Vector store similarity search will produce "k" documents (in our example "k" = 3)
 - We don't need all the retrieve schemas, so We'll filter to get only relevant schemas

In [52]:
class GradeDocument(BaseModel):
    score: str = Field(
        description="Document is relevant to the question? If yes -> 'Yes' if not -> 'No'"
    )

In [137]:
llm = init_chat_model("google_genai:gemini-2.0-flash")
#llm = ChatOllama(model="qwen2.5-coder:7b", temperature=0.1)

In [138]:
def retrieval_grader(user_query: str):
    print("Entering retrieval_grader")
    system_message = SystemMessage(
        content="""You are a grader assessing the relevance of a retrieved document to a user question.
Only answer with 'Yes' or 'No'.

If the document contains information relevant to the user's question, respond with 'Yes'.
Otherwise, respond with 'No'."""
    )

    structured_llm = llm.with_structured_output(GradeDocument)

    relevant_docs = []
    for doc in schema_docs:
        human_message = HumanMessage(
            content=f"User question: {user_query}\n\nRetrieved document:\n{doc.page_content}"
        )
        grade_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
        grader_llm = grade_prompt | structured_llm
        result = grader_llm.invoke({})
        print(
            f"Grading document: {doc.page_content[:30]}... Result: {result.score.strip()}"
        )
        if result.score.strip().lower() == "yes":
            relevant_docs.append(doc)

    if len(relevant_docs) > 0:
        return relevant_docs
    else:
        return schema_docs

In [139]:
user_input

'show the total taxi revenue'

In [140]:
relevant_docs = retrieval_grader(user_input)
relevant_docs

Entering retrieval_grader
Grading document: Table VW_TAXI_COLLECTION (It c... Result: Yes
Grading document: Table VW_FM_NRTDETAILS (It con... Result: No
Grading document: Table VW_EMPLOYEE_OUTSTANDING ... Result: No


[Document(id='9b0c378d-f49e-46e1-8a96-8dfaef2f88e5', metadata={}, page_content='Table VW_TAXI_COLLECTION (It contains the taxi or driver collecton and driver revenue information as date wise.\nSo, for a specific range we can get the revenue.): COMPCODE, CASHIER_NAME, PAID_DATE, DRIVER_ID, LOCATION_NAME, AMOUNT, TAXI_REVENUE, CALLTRIP, SURCHARGE_ITC, SURCHARGE_DRIVER, CREDIT_CARD_AMOUNT, ACTUAL_REVENUE, DRIVER_INCOME, SHFTCODE')]